# Description

* This is the one used


# Setting variables

In [1]:
import os

## What is the base directory?
baseDir = '/home/sam/notebooks/hemp_microbiome/data/ITS_OTUs/'
## What directory do you want to work in and keep all subsequent files in?
workDir = os.path.join(baseDir, "OTU_binning")


## Where is your final QC'ed and ITSxed sequence fasta file?
#qcFinal = os.path.join(workDir, 'QC', 'ITSx', 'main.rename.fasta')
qcFinal = os.path.join(baseDir, 'QC', 'finalQC.fasta')

## What database do you want to use for UCHIME chimera filtering?
UC_db = '/home/sam/databases/UNITE/uchime_reference_dataset_28.06.2017/ITS1_ITS2_datasets/uchime_reference_dataset_ITS1_28.06.2017.fasta'

## What database do you want to base your taxonomy on when using USEARCH SINTAX?
#US_dbDir = '~/databases/UNITE/utax_reference_dataset_01.12.2017.fasta'
#US_dbSeqs = os.path.join(US_dbDir, 'refdb.udb')
US_dbDir = '/home/sam/databases/UNITE/sintaxDB'
US_dbSeqs = os.path.join(US_dbDir, 'utax_reference_dataset_01.12.2017.fasta')

nprocs = 10

# Init

In [2]:
import sys
import glob
import pandas as pd
from Bio import SeqIO
import multiprocessing
import uuid
%load_ext rpy2.ipython


In [3]:
%%R
library(ggplot2)
library(dplyr)
library(tidyr)
library(gridExtra)

/home/sam/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/home/sam/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/home/sam/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)
/home/sam/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 
Attaching package: ‘gridExtra’


  warnings.warn(x, RRuntimeWarning)
/home/sam/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following object is masked from ‘package:dplyr’:

    combine


  warnings.warn(x, RRuntimeWarning)


In [4]:
if not os.path.isdir(workDir):
    print ("Working directory does not exist. Making it now")
    os.makedirs(workDir)

%cd $workDir

/home/sam/notebooks/hemp_microbiome/data/ITS_OTUs/OTU_binning


### Symlinking qc'ed seq file

In [5]:
!ln -s $qcFinal .

In [6]:
qcFinal = os.path.split(qcFinal)[1]
!ls -thlc $qcFinal

lrwxrwxrwx 1 sam sam 66 May 29 10:20 finalQC.fasta -> /home/sam/notebooks/hemp_microbiome/data/ITS_OTUs/QC/finalQC.fasta


# Full Derepication sequences

In [7]:
FullDerepFile = os.path.splitext(qcFinal)[0] + '.unique.fasta'
FullDerepUC = os.path.splitext(qcFinal)[0] + '.unique.uc'

!vsearch --threads $nprocs \
    --derep_fulllength $qcFinal \
    --minuniquesize 2 \
    --sizein \
    --sizeout \
    --fasta_width 0 \
    --uc $FullDerepUC \
    --output $FullDerepFile

vsearch v2.3.4_linux_x86_64, 125.8GB RAM, 40 cores
https://github.com/torognes/vsearch

Reading file finalQC.fasta 100%                                                                                                                                                                                                                                                                                    
2531893124 nt in 10252187 seqs, min 50, max 432, avg 247
Dereplicating 100%                                                                                                                                                                                                                                      
Sorting 100%
1147070 unique sequences, avg cluster 8.9, median 1, max 4768318
Writing output file 100%      
Writing uc file, first part 100%                                
Writing uc file, second part 100%                
150007 uniques written, 997063 clusters discarded (86.9%)


In [8]:
ret = !grep -c ">" $FullDerepFile
print ('Number of unique sequences: {}'.format(ret[0]))

Number of unique sequences: 150007


In [10]:
!head -n 10 $FullDerepFile

>2-3-L_4;size=4768318;
TTGTCGAAACCTGCAACAGCAGAACGACCCGTGAACACGTTTTAAACAGCTTGGGCGGGCGAGAGGAGCTTGCTCCTTGGACCCGCCCGCACCTGCTGGGAGAAATCTCGGCGGGCTAACGAACCCCGGCGCAATCTGCGCCAAGGAACAATAAAAGATTATCGCGTGGCTCGTGCGGTGGCCCGGAGACGGTGTCCGCCAATCGAGATGCGTGTTTATCGAAATGTCTAAACGACTCTCGGCAACGGATATCTCGGCTCTCGC
>4-1-RS_39;size=379484;
TTGTCGAAACCTGCAACAGCAGAACGACCCGTGAACACGTTTTAAACAGCTTGGGCGGGCGAGAGGAGCTTGCTCCTTGGACCCGCCCGCACCTGCTGGGAGAAATCTCGGCGGGCTAACGAACCCCGGCGCAATCTGCGCCAAGGATCAATAAAAGATTATCGCGTGGCTCGTGCGGTGGCCCGGAGACGGTGTCCGCCAATCGAGATGCGTGTTTATCGAAATGTCTAAACGACTCTCGGCAACGGATATCTCGGCTCTCGC
>1-4-RS_84;size=298542;
TTGTCGAAACCTGCAACAGCAGAACGACCCGTGAACACGTTTTAAACAGCTTGGGCGGGCGAGAGGAGCTTGCTCCTTGGACCCGCCCGCACCTGCTGGGAGAAATCTCGGCGGGCTAACGAACCCCGGCGCAATCTGCGCCAAGGAACAATAAAAGATTATCGCGTGGCTCGTGCGGTGGCCCGGAGACGGTGTCCGCCAAGCGAGATGCGTGTTTATCGAAATGTCTAAACGACTCTCGGCAACGGATATCTCGGCTCTCGC
>F3-2_78;size=138547;
TTGTCGAAACCTGCAACAGCAGAACGACCCGTGAACACGTTTTAAACAGCTTGGGCGGGCGAGAGGAGCTTGCTCCTTGGACCCGCCCGCACCTGCTGGGAGAAATCTCGGC

In [12]:
!head -n 10 $FullDerepUC

S	0	264	*	*	*	*	*	2-3-L_4	*
H	0	264	100.0	+	0	0	*	3-1-RT_27	2-3-L_4
H	0	264	100.0	+	0	0	*	1-5-L_45	2-3-L_4
H	0	264	100.0	+	0	0	*	4-1-RT_49	2-3-L_4
H	0	264	100.0	+	0	0	*	4-5-F_57	2-3-L_4
H	0	264	100.0	+	0	0	*	3-4-F_60	2-3-L_4
H	0	264	100.0	+	0	0	*	3-5-F_63	2-3-L_4
H	0	264	100.0	+	0	0	*	3-1-F_64	2-3-L_4
H	0	264	100.0	+	0	0	*	4-4-RT_70	2-3-L_4
H	0	264	100.0	+	0	0	*	5-2-L_75	2-3-L_4


# Keep just ITS1 region
This removes mainly part of the 5.8S

In [13]:
def ITSx_func(filename):
    workername = str(multiprocessing.Process())
    workername = str(uuid.uuid3(uuid.NAMESPACE_DNS, workername))
    ITS1_file = workername + '.ITS1.partial_output.fasta'
    noDet_file = workername + '.no_detections.partial_output.fasta'
    subfastafile = workername + '.sub.fasta'
    ITSx_output = workername + '.ITSx_output'
    ITSx_output_ITS1 = ITSx_output + '.ITS1.fasta'
    ITSx_output_noDet = ITSx_output + '_no_detections.fasta'
    remove_output = ITSx_output + '*'
    
    !touch $ITS1_file
    !touch $noDet_file
    
    fasta_sequences = SeqIO.parse(open(filename),'fasta')
    for fasta in fasta_sequences:
        SeqIO.write(fasta, subfastafile, "fasta")
        !ITSx -i $subfastafile \
            -o $ITSx_output \
            --save_regions ITS1 \
            --cpu 1 \
            --preserve T \
            --reset F \
            -t all \
            --heuristics \
            --table F \
            --silent T \
            --detailed_results F \
            --graphical F
        !cat $ITSx_output_ITS1 >> $ITS1_file
        !cat $ITSx_output_noDet >> $noDet_file
        !rm $remove_output
        !rm $subfastafile

In [14]:
ITSxdir = os.path.join(workDir, "ITSx_output")

if not os.path.isdir(ITSxdir):
    print ("ITSx directory does not exist. Making it now")
    os.makedirs(ITSxdir)

%cd $ITSxdir

ITSx directory does not exist. Making it now
/home/sam/notebooks/hemp_microbiome/data/ITS_OTUs/OTU_binning/ITSx_output


In [16]:
FullDerepFile = os.path.join(workDir, FullDerepFile)
!fasta-splitter.pl \
    --n-parts $nprocs \
    $FullDerepFile
    


/home/sam/notebooks/hemp_microbiome/data/ITS_OTUs/OTU_binning/finalQC.unique.fasta: 150007 sequences, 33210565 bp => dividing into 10 parts .......... OK
All done, 2 seconds elapsed


In [ ]:
partialfastalist = [f for f in os.listdir(ITSxdir) if 'finalQC.unique.part-' in f]


pool = multiprocessing.Pool(processes=nprocs)

if __name__ == '__main__':
    # calculate the chunk size as an integer
    pool.map(ITSx_func, partialfastalist)
    pool.close()
    pool.terminate()
    pool.join()


ITSx -- Identifies ITS sequences and extracts the ITS region
by Johan Bengtsson-Palme et al., University of Gothenburg
Version: 1.1b1
-----------------------------------------------------------------
ITSx -- Identifies ITS sequences and extracts the ITS region
by Johan Bengtsson-Palme et al., University of Gothenburg
Version: 1.1b1
-----------------------------------------------------------------
ITSx -- Identifies ITS sequences and extracts the ITS region
by Johan Bengtsson-Palme et al., University of Gothenburg
Version: 1.1b1
-----------------------------------------------------------------
ITSx -- Identifies ITS sequences and extracts the ITS region
by Johan Bengtsson-Palme et al., University of Gothenburg
Version: 1.1b1
-----------------------------------------------------------------
ITSx -- Identifies ITS sequences and extracts the ITS region
by Johan Bengtsson-Palme et al., University of Gothenburg
Version: 1.1b1
-----------------------------------------------------------------


In [18]:
!cat *.no_detections.partial_output.fasta > no_ITS_detections.fasta
!cat *.ITS1.partial_output.fasta > ITS1_sequences.fasta

In [35]:
#!rm *.no_detections.partial_output.fasta
#!rm *.ITS1.partial_output.fasta

In [19]:
!grep -c ">" ITS1_sequences.fasta

126245


In [20]:
!head ITS1_sequences.fasta

>4-1-L_6689723;size=2;
TTACCTAGAGTTGTAGGCTTTGCCTGCTATCTCTTACCCATGTCTTTTGAGTACCTTCGTTTCCTCGGCGGGTCCGCCCGCCGATTGGACAATTTAAACCATTTGCAGTTGCAATCAGCGTCTGAGAAAACTCAATAGTTA
>4-1-L_6710936;size=2;
TTGTCGAAACCTGCAACAGCAAAACGACCCGTGAACACGTTTTAAACAGCTTGGGCGGGCGAGAGGAGCTTGCTCCTTGGACCCGCCCGCACCTGCTGGGAGAAATCTCGGCGGGCTAATGAACCCCGGCGCAATCTGCGCCAAGGAACAATAAAAGATTATCGCGTGGCTCGTGCGGTGGCCCGGAGACGGTGTCCGCCAATCGAGATGCGTGTTTATCGAAATGTCTAAA
>4-1-L_672628;size=2;
TTACCTAGAGTTTGTGGACTTCGGTCTGCTACCTCTTACCCATGTCTTTTGAGTACCTTCGTCTCCTCGGCGGGTCCGCCCGCCGATTGGACAACATTCAAACCCTTTGCAGTTGCAATCAGCGTCTGAAAAAACATAATAGTTA
>4-1-L_685618;size=2;
TTGTCGAAACATGCAACAGCATAACGACCCGTGAACACGTTTTAAACAGCTTGGGCGGGCGAGAGGAGCTTGCTCCTTGGACCCGCCCGCACCTGCTGGGAGAAATCTCGGCGGGCTAACGAACCCCGGCGCAATCTGCGCCAAGGATCAATAAAAGATTATCGCGTGGCTCGTGCGGTGGCCCGGAGACGGTGTCCGCCAATCGAGATGCGTGTTTATCGAAATGTCTAAA
>4-1-L_6884476;size=2;
TTACACAAATATGCAGGCGGGCTGGAACCCCTCGGGGTTACAGCCTTGCTGAATTATTCACCCTTGTCTTTTGCGTACTTCTTGTTTCCTTGGTGGGTTCGCCCACCACTAGGACAAACATAAACCTTTTGTA

# Add N's to the end of the sequence for help with clustering

In [21]:
%cd $workDir

/home/sam/notebooks/hemp_microbiome/data/ITS_OTUs/OTU_binning


In [22]:
ITSx_final = os.path.join(ITSxdir, 'ITS1_sequences.fasta')

In [23]:
!python2 /home/sam/repo/ITS_trim.py \
    -F $ITSx_final \
    -d $workDir \
    -f fasta


# Precluster sequences at 98%

In [24]:
trimmedUniqITSx = os.path.join(workDir, 'globtrim.fasta')

In [25]:
qcFinalUniqPreC = os.path.splitext(qcFinal)[0] + '.preclustered.fasta'
PreCucfile = os.path.splitext(qcFinal)[0] + '.preclustered.uc'

!vsearch --threads $nprocs\
    --cluster_size $trimmedUniqITSx \
    --id 0.98 \
    --strand plus \
    --sizein \
    --sizeout \
    --fasta_width 0 \
    --uc $PreCucfile \
    --centroids $qcFinalUniqPreC

vsearch v2.3.4_linux_x86_64, 125.8GB RAM, 40 cores
https://github.com/torognes/vsearch

Reading file /home/sam/notebooks/hemp_microbiome/data/ITS_OTUs/OTU_binning/globtrim.fasta 100%               
45069465 nt in 126245 seqs, min 357, max 357, avg 357
Masking 100%                                            
Sorting by abundance 100%
Counting unique k-mers 100%       
Clustering 100%                                                                                                                                                                                                                                                                                                                                                                                                          
Sorting clusters 100%
Writing clusters 100%   
Clusters: 4219 Size min 2, max 7590187, avg 29.9
Singletons: 0, 0.0% of seqs, 0.0% of clusters


In [26]:
ret = !grep -c ">" $qcFinalUniqPreC
print ('Number of 98% clustered sequences: {}'.format(ret[0]))

Number of 98% clustered sequences: 4219


In [27]:
!tail -n 10 $qcFinalUniqPreC

>M5-1_3044292;size=2;
TTACAGAGCGAGAGGCTCAGCCATGACGGTAGTCGTGTGCTGGGTCGACCCTCCCACCCGTGTCGATATGTATTTTGTTGCTTTGGCGGGCCGGGTTTGCGCCCCGTTCGGCGTCCGCTGTGGCGCTGGGGGAGCCGCCCACCGGCTTCGGCTGGAGCGTGCCCGCCAGAGACCTCATCCAAACTCACGTTGTCATGTAGTCTAAGCTTTATTAAGTAATTGATannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
>M5-2_10323638;size=4;
TTATTGAATGTATTTGCAAGTTGGTTGTAGCTGGCTCTTCGGAGTAATGTGCTCGCCACTTGTCTCTACTACTTTTCCACCTGTGAACCTTTGTAGATCACGAATGAACTCTCGCCTGAAAATGGCGGTTTGAGGGACTTGTCCCTTGATAGTTCGTGGTCTATGTTTTATAATCTACTCTATGTTTGTCTTTGAATGTCTTTTTAAATGGTCTTTGACCTTTAAATACTATannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
>M5-2_626442;size=2;
TTACAGAGTTTAACGACTCCCAAACCACTGTGAACATACCCGTACCGTTGCCTCGGCGGGCGGCCCCAGGGCGGGGCCGCAGCCTCCCCAGCGGAGGCGCCCGCCGCAGGTCGCAAAACTATAACTATATTTAGTGGCATCTCTGAGTAACTTCCAAACAATCannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn

# Filtering chimeras

### De novo chimera filtering

In [28]:
noChimera1 = os.path.splitext(qcFinal)[0] + '.denovo.nonchimeras.fasta'

!vsearch --threads $nprocs \
    --uchime_denovo $qcFinalUniqPreC \
    --sizein \
    --sizeout \
    --fasta_width 0 \
    --nonchimeras $noChimera1

vsearch v2.3.4_linux_x86_64, 125.8GB RAM, 40 cores
https://github.com/torognes/vsearch

Reading file finalQC.preclustered.fasta 100%  
1506183 nt in 4219 seqs, min 357, max 357, avg 357
Masking 100%               
Sorting by abundance 100%
Counting unique k-mers 100%  
Detecting chimeras 100%                                                                                                                                                                                                          
Found 140 (3.3%) chimeras, 4072 (96.5%) non-chimeras,
and 7 (0.2%) borderline sequences in 4219 unique sequences.
Taking abundance information into account, this corresponds to
2400 (0.0%) chimeras, 9048330 (99.7%) non-chimeras,
and 22010 (0.2%) borderline sequences in 9072740 total sequences.


In [29]:
ret = !grep -c ">" $noChimera1
print ('Number of sequences after de novo chimera detection: {}'.format(ret[0]))

Number of sequences after de novo chimera detection: 4072


### Chimera filtering based on references

In [30]:
noChimera2 = os.path.splitext(qcFinal)[0] + '.ref.nonchimeras.fasta'

!vsearch --threads $nprocs \
    --uchime_ref $noChimera1 \
    --db $UC_db \
    --sizein \
    --sizeout \
    --fasta_width 0 \
    --nonchimeras $noChimera2

vsearch v2.3.4_linux_x86_64, 125.8GB RAM, 40 cores
https://github.com/torognes/vsearch

Reading file /home/sam/databases/UNITE/uchime_reference_dataset_28.06.2017/ITS1_ITS2_datasets/uchime_reference_dataset_ITS1_28.06.2017.fasta 100%  
6677951 nt in 30523 seqs, min 66, max 1958, avg 219
Masking 100%        
Counting unique k-mers 100%   
Creating index of unique k-mers 100%         
Detecting chimeras 100%                
Found 13 (0.3%) chimeras, 4057 (99.6%) non-chimeras,
and 2 (0.0%) borderline sequences in 4072 unique sequences.
Taking abundance information into account, this corresponds to
1998 (0.0%) chimeras, 9046309 (100.0%) non-chimeras,
and 23 (0.0%) borderline sequences in 9048330 total sequences.


In [31]:
ret = !grep -c ">" $noChimera2
print ('Number of sequences after reference based chimera detection: {}'.format(ret[0]))

Number of sequences after reference based chimera detection: 4057


In [32]:
!head $noChimera2

>6-3-BS_1438;size=45752;
TTGAATATGCAAACCACAACACGCACTCTCTACTTGTGTCTGACGTTAGGTCAACATGCGCAAGCATGTGGCCTACTAACCTACACACCATTGACCAACCAATTATCAAACCAAACTCTGAAGCTTTGGCTGCTGTTAATCGGCAGTTTTAACAAAGaannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
>2-2-F_439;size=94950;
TTAGTGATTGACCTCCGGGTCTTAATAACTATAATCTTCTACCTCTGTGAACCGTTGGTCTTCGGACCGATTCTTATAACAAACATCTGTGTAATGAACGTAACCTATATTAATCATATannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
>3-4-L_689;size=322641;
TTAGTGAATACACCGGGGAACCCACTCTGTGGGCCCCAACCCTTCACCATATCCTCACACCTGTGCACCGCTGGTAGATGCCGCTGTACTTCGGTACGGTTTGCAGCTGCTGGCAACCTATACACACACACCTGAGTTCTATGAATGTTAATTTTACTTGCCTTCGGGCAGAACTaaaatannnnnnnnnnnnnnnnnnnnnnnnnnnnnnn

# Readmapping and Reinflating

Reinflating from full dereplication

In [33]:
print(trimmedUniqITSx)
print(PreCucfile)
print(noChimera2)

/home/sam/notebooks/hemp_microbiome/data/ITS_OTUs/OTU_binning/globtrim.fasta
finalQC.preclustered.uc
finalQC.ref.nonchimeras.fasta


In [34]:
readmap1 = 'final.nonchimeras.derep.fasta'
!perl /home/sam/repo/map.pl \
    $trimmedUniqITSx \
    $PreCucfile \
    $noChimera2 > $readmap1

In [35]:
ret = !grep -c ">" $readmap1
print ('Number of unique non-chimeric sequences: {}'.format(ret[0]))

Number of unique non-chimeric sequences: 125081


In [36]:
!head $readmap1

>4-1-L_6689723;size=2;
TTACCTAGAGTTGTAGGCTTTGCCTGCTATCTCTTACCCATGTCTTTTGAGTACCTTCGTTTCCTCGGCGGGTCCGCCCGCCGATTGGACAATTTAAACCATTTGCAGTTGCAATCAGCGTCTGAGAAAACTCAATAGTTANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
>4-1-L_6710936;size=2;
TTGTCGAAACCTGCAACAGCAAAACGACCCGTGAACACGTTTTAAACAGCTTGGGCGGGCGAGAGGAGCTTGCTCCTTGGACCCGCCCGCACCTGCTGGGAGAAATCTCGGCGGGCTAATGAACCCCGGCGCAATCTGCGCCAAGGAACAATAAAAGATTATCGCGTGGCTCGTGCGGTGGCCCGGAGACGGTGTCCGCCAATCGAGATGCGTGTTTATCGAAATGTCTAAANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
>4-1-L_672628;size=2;
TTACCTAGAGTTTGTGGACTTCGGTCTGCTACCTCTTACCCATGTCTTTTGAGTACCTTCGTCTCCTCGGCGGGTCCGCCCGCCGATTGGACAACATTCAAACCCTTTGCAGTTGCAATCAGCGTCTGAAAAAACATAATAGTTANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

Reinflating from per sample dereplication. Do this in the in Reinflation.ipynb

In [54]:
readmap2 = 'final.nochimeras.fasta'


ret = !grep -c ">" $readmap2
print ('Number of total non-chimeric sequences: {}'.format(ret[0]))

Number of total non-chimeric sequences: 9046309


In [55]:
!head $readmap2

>4-1-L_6689723;sample=4-1-L;
TTACCTAGAGTTGTAGGCTTTGCCTGCTATCTCTTACCCATGTCTTTTGAGTACCTTCGTTTCCTCGGCGGGTCCGCCCGCCGATTGGACAATTTAAACCATTTGCAGTTGCAATCAGCGTCTGAGAAAACTCAATAGTTANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
>4-1-L_6689808;sample=4-1-L;
TTACCTAGAGTTGTAGGCTTTGCCTGCTATCTCTTACCCATGTCTTTTGAGTACCTTCGTTTCCTCGGCGGGTCCGCCCGCCGATTGGACAATTTAAACCATTTGCAGTTGCAATCAGCGTCTGAGAAAACTCAATAGTTANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
>4-1-L_6710936;sample=4-1-L;
TTGTCGAAACCTGCAACAGCAAAACGACCCGTGAACACGTTTTAAACAGCTTGGGCGGGCGAGAGGAGCTTGCTCCTTGGACCCGCCCGCACCTGCTGGGAGAAATCTCGGCGGGCTAATGAACCCCGGCGCAATCTGCGCCAAGGAACAATAAAAGATTATCGCGTGGCTCGTGCGGTGGCCCGGAGACGGTGTCC

# Cluster OTUs at 97%

In [56]:
otu_file = 'final.otu.fasta'
otu_ucfile = 'final.otu.uc'
otutabout_file = 'final.otutab.txt'

!vsearch --threads $nprocs \
    --cluster_size $readmap2 \
    --id 0.97 \
    --strand plus \
    --fasta_width 0 \
    --uc $otu_ucfile \
    --relabel OTU_ \
    --centroids $otu_file \
    --otutabout $otutabout_file

vsearch v2.3.4_linux_x86_64, 125.8GB RAM, 40 cores
https://github.com/torognes/vsearch

Reading file final.nochimeras.fasta 100%                                                                                                                                                                                                                                                                                                                                                                                                          
3229532313 nt in 9046309 seqs, min 357, max 357, avg 357
Masking 100%                                                                                                                                                                                                                                                                                                                                                                                                                
Sorting

In [57]:
ret = !grep -c ">" $otu_file
print ('Number of final OTUs: {}'.format(ret[0]))

Number of final OTUs: 3737


In [58]:
!head $otu_file

>OTU_1
TTGTCGAAACCTGCAACAGCAGAACGACCCGTGAACACGTTTTAAACAGCTTGGGCGGGCGAGAGGAGCTTGCTCTTGGACCCGCCCGCACCTGCTGGGAGAAATCTCGGCGGGCTAACGAACCCCGGCGCAATCTGCGCCAAGGAACAATAAAAGATTATCGCGTGGCTCGTGCGGTGGCCCGGAGACGGTGTCCGCCAATCGAGATGCGTGTTTATCGAAATGTCTaaannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
>OTU_2
TTACAGGACTCGCAAGACTCCTACAACCCTGTGAACCTACTATACGAACGTTGCTTCGGCGGGTGCGCCCCGGGGATTCGTTCCCGCGGCGCGTCAGCCCGCCGGCAGCCTACAAATTCTGTTTCTTTGCGTTGGCATCTCGAGAAAAACAAATAAGTTannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
>OTU_3
TTACTGAGTTTTTAACTCTCCAAACCATGTGAACTTACCACTGTTGCCTCGGTGGTTGGTCTTCGGACTGACCACCGGTGGACTACTAAACTCTTGTTAATTTTTGGCATTCTGAATCATAACTAAGAAATAAGTTannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn

In [59]:
!tail $otu_file

>OTU_3733
TTAACGAGTTTTTGAAAGGGGTTGTAGCTGGCCAAAAACAAAGGCATGTGCACACCCTGCTCAATTCCACTCTCACACCCCTGTGCACTTACTGTAGGCTATGGTCGGATAGCCAACTTCGGTTGGTTTGAAAGCCTTGCTTATGTTATTACAAACGCTTCAGTTAAAGAATGTCATACCGCGTATAACGCATTAAATannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
>OTU_3734
TTAGTGAATTTGGCCTCTGGCCTCATTAACACAAACACACCGTGAACCGTGGCTTCGGCCCAAACTTTTATGTATTGAATGTAATATATCATAACaaaaacannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
>OTU_3735
TTATCGAGTCCAATCTAGCTCATTTCTGGCTGTGCATTCACTTGCACGCCGGGGCGAGTTGGCTCTTGAACTTCTCACACCTGTGCACCTTCTCGTGCCTTCTCCGTCGTCGCCCTTTGTTGGGAGACTCCACCGCTTGCGGTGGGATTCGGGCGTTGCAGTAAATTTGCTTTTACGGCTCGCCTCGGGGAGAGGGCACTTGCTTTTTTACACCTTTTTTGTATGTCTCAGAATGTCTTGGATTGCCCTGTGAT

In [60]:
# Remove N's
otu_file_noN = 'otu_noN.fasta'
!sed -e 's/n//g' $otu_file > $otu_file_noN

In [61]:
!head $otu_file_noN

>OTU_1
TTGTCGAAACCTGCAACAGCAGAACGACCCGTGAACACGTTTTAAACAGCTTGGGCGGGCGAGAGGAGCTTGCTCTTGGACCCGCCCGCACCTGCTGGGAGAAATCTCGGCGGGCTAACGAACCCCGGCGCAATCTGCGCCAAGGAACAATAAAAGATTATCGCGTGGCTCGTGCGGTGGCCCGGAGACGGTGTCCGCCAATCGAGATGCGTGTTTATCGAAATGTCTaaa
>OTU_2
TTACAGGACTCGCAAGACTCCTACAACCCTGTGAACCTACTATACGAACGTTGCTTCGGCGGGTGCGCCCCGGGGATTCGTTCCCGCGGCGCGTCAGCCCGCCGGCAGCCTACAAATTCTGTTTCTTTGCGTTGGCATCTCGAGAAAAACAAATAAGTTa
>OTU_3
TTACTGAGTTTTTAACTCTCCAAACCATGTGAACTTACCACTGTTGCCTCGGTGGTTGGTCTTCGGACTGACCACCGGTGGACTACTAAACTCTTGTTAATTTTTGGCATTCTGAATCATAACTAAGAAATAAGTTa
>OTU_4
TTGAATATGCAAACCACAACACGCACTCTCTACTTGTGTCTGACGTTAGGTCAACGTGCGCAAGCATGTGGCCTACTAACCTACACACCATTGACCAACCAATTATCAAACCAAACTCTGAAGCTTTGGCTGCTGTTAATCGGCAGTTTTAACAAAGaa
>OTU_5
TTACCGAGTGCGGGCACTCGCGGCCCAACCTCCCACCCTTGTCTCTATACACCTGTTGCTTTGGCGGGCCCACCGGGGCCACCTGGTCGCCGGGGGACGCACGTCCCCGGGCCCGCGCCCGCCGAAGCGCTCTGTGAACCCTGATGAAGATGGGCTGTCTGAGTACTATGAAAATTGTCa


In [62]:
!head $otutabout_file

#OTU ID	1-1-BS	1-1-F	1-1-L	1-1-RS	1-1-RT	1-2-BS	1-2-F	1-2-L	1-2-RS	1-2-RT	1-3-BS	1-3-F	1-3-L	1-3-RS	1-3-RT	1-4-BS	1-4-F	1-4-L	1-4-RS	1-4-RT	1-5-BS	1-5-F	1-5-L	1-5-RS	1-5-RT	2-1-BS	2-1-F	2-1-L	2-1-RS	2-1-RT	2-2-BS	2-2-F	2-2-L	2-2-RS	2-2-RT	2-3-BS	2-3-F	2-3-L	2-3-RS	2-3-RT	2-4-BS	2-4-F	2-4-L	2-4-RS	2-4-RT	2-5-BS	2-5-F	2-5-L	2-5-RS	2-5-RT	3-1-BS	3-1-F	3-1-L	3-1-RS	3-1-RT	3-2-BS	3-2-F	3-2-L	3-2-RS	3-2-RT	3-3-BS	3-3-F	3-3-L	3-3-RS	3-3-RT	3-4-BS	3-4-F	3-4-L	3-4-RS	3-4-RT	3-5-BS	3-5-F	3-5-L	3-5-RS	3-5-RT	4-1-BS	4-1-F	4-1-L	4-1-RS	4-1-RT	4-2-BS	4-2-F	4-2-L	4-2-RS	4-2-RT	4-3-BS	4-3-F	4-3-L	4-3-RS	4-3-RT	4-4-BS	4-4-F	4-4-L	4-4-RS	4-4-RT	4-5-BS	4-5-F	4-5-L	4-5-RS	4-5-RT	5-1-BS	5-1-F	5-1-L	5-1-RS	5-1-RT	5-2-BS	5-2-F	5-2-L	5-2-RS	5-2-RT	5-3-BS	5-3-F	5-3-L	5-3-RS	5-3-RT	5-4-BS	5-4-F	5-4-L	5-4-RS	5-4-RT	5-5-BS	5-5-F	5-5-L	5-5-RS	5-5-RT	6-1-BS	6-1-F	6-1-L	6-1-RS	6-1-RT	6-2-BS	6-2-F	6-2-L	6-2-RS	6-2-RT	6-3-BS	6-3-F	6-3-L	6-3-RS	6-3-RT	6-4-BS	6-4-F	6-4-L	6-4-RS	6-4-RT	6-5-BS	6-5-F	6-5-L	6-5-RS	6-5-RT	F1

# Adding taxonomy

### Taxonomy with vsearch sintax

In [63]:
!head $US_dbSeqs


>LC146734|SH497095.07FU;tax=d:Fungi,p:Basidiomycota,c:Agaricomycetes,o:Agaricales,f:Tricholomataceae,g:Flagelloscypha,s:Flagelloscypha_japonica_SH497095.07FU;
AGGTGAACCTGCGGAAGGATCATTATTGAATTTCACCGGTTTGGTCTGTTGCTGGTTCCGAAAGGTTCATGTGCACGCCTCGCCTCTGATATCTCACCACCTGTGAACTTTAGTGGGCTGTGACGGCCTTGTCTTCAGCAGTTTCGTGTTGGGTTTTGGGTACTTGTACCTGCCAATGCGCTCTGCTGGGGGGAGGTTTAGTCCAGTCTGCGCCCTTTTACACACTCTATTGTATGTCTAGAAAGTCCATTATCGGTCGGAAGACCTTTAATCTTACACAACTATTGACAACGGATCTCTTGGCTCACGCATCGATGAAGAACGCAGCGAAATGCGATAAGTAATGTGAATTGCAGATTTCAGTGAATCATCGAATCTTTGAACGCACCTTGCACCTTCTGGTATTCCGGAAGGTACATCTGTTTGAGTGTCATTAATCTCTCAACTCTACATGTTTTATTATGTGTATGTGTTGGTTGTGAGGGCTTGCGGGCGTCCTTTCAGTAGGATGCCAGCTCCCTTGAAATGCATTAGCTGGGAACTAGTACATGGGTTTGAGTGCTTCGATGCCTGAAGACCTCCTGGTGTGATAATTATCTACGCCGTTGGTTTAGTCGTTGCGCAACTGTGGAACTCTTTCTGCTTCTAACTGTCTCGCAAGGGACTAGTCCGCTTCAGGGCTACTTGACTAATGTGACCTCAAATCAGATGGGACTACCCGCTGAACTTAAGCATATCAATAAGCGGAGGAAAAGAAACTAACAAGGATTCCCCTAGTAACTGCGAGTGAAGCGGGAAAAGCTCAAATTTGAAATCTGGCTGTCGTATGGCAGTCCGAGTT

In [64]:
taxfile = "otu_tax_Vsintax_unite.txt"
!/home/sam/repo/vsearch-2.8.0/bin/vsearch --sintax $otu_file_noN \
    --db $US_dbSeqs \
    --tabbedout $taxfile \
    --sintax_cutoff 0.8 \
    --threads $nprocs

vsearch v2.8.0_linux_x86_64, 125.8GB RAM, 40 cores
https://github.com/torognes/vsearch

Reading file /home/sam/databases/UNITE/sintaxDB/utax_reference_dataset_01.12.2017.fasta 100%   
42413580 nt in 58049 seqs, min 216, max 7491, avg 731
Counting k-mers 100%          
Creating k-mer index 100%                                                
Classifying sequences 100%                                                                                                                          
Classified 1663 of 3737 sequences (44.50%)


In [65]:
!head $taxfile

OTU_4				
OTU_1	d:Plantae(1.00)	+	d:Plantae
OTU_10				
OTU_6				
OTU_2	d:Fungi(1.00),p:Ascomycota(0.98),c:Sordariomycetes(0.98),o:Chaetosphaeriales(0.98),f:Chaetosphaeriaceae(0.98),g:Chloridium(0.98)	+	d:Fungi,p:Ascomycota,c:Sordariomycetes,o:Chaetosphaeriales,f:Chaetosphaeriaceae,g:Chloridium
OTU_3	d:Fungi(1.00),p:Ascomycota(1.00),c:Sordariomycetes(1.00),o:Xylariales(1.00),f:Microdochiaceae(1.00),g:Microdochium(1.00)	+	d:Fungi,p:Ascomycota,c:Sordariomycetes,o:Xylariales,f:Microdochiaceae,g:Microdochium
OTU_7	d:Fungi(1.00),p:Ascomycota(1.00),c:Sordariomycetes(1.00),o:Hypocreales(1.00),f:Hypocreaceae(1.00),g:Trichoderma(1.00),s:Trichoderma_evansii_SH327536.07FU(0.81)	+	d:Fungi,p:Ascomycota,c:Sordariomycetes,o:Hypocreales,f:Hypocreaceae,g:Trichoderma,s:Trichoderma_evansii_SH327536.07FU
OTU_8	d:Fungi(1.00),p:Ascomycota(0.96),c:Sordariomycetes(0.96),o:Hypocreales(0.96),f:Nectriaceae(0.96),g:Fusarium(0.96),s:Fusarium_oxysporum_SH289658.07FU(0.58)	+	d:Fungi,p:Ascomycota,c:Sordariomycetes,o:H

In [66]:
!grep -c "d:Fungi" $taxfile

1532


In [67]:
!grep -c "OTU" $taxfile

3737


In [19]:
# make sure no existing biome file
otu_biom_file = 'otu_table.biom'
otuTax = os.path.join('sintax_output', 'sintax_table_unite_97_otu_reformatted.txt')

otu_biom_tax_file = 'otu_table_wtax.biom'
if os.path.isfile(otu_biom_tax_file):
    os.unlink(otu_biom_tax_file)
    
# bio add-metadata
cmd = """biom add-metadata -i {} -o {} \
    --observation-metadata-fp {} \
    --sc-separated taxonomy \
    --float-fields consensus \
    --int-fields numhits \
    --observation-header OTUID,taxonomy,consensus,numhits"""
cmd = cmd.format(otu_biom_file, otu_biom_tax_file, otuTax)
!$cmd

# Final files

In [68]:
print ('OTU table: {}'.format(otutabout_file))
print ('OTU file: {}'.format(otu_file_noN))
print ('Taxonomy file: {}'.format(taxfile))

OTU table: final.otutab.txt
OTU file: otu_noN.fasta
Taxonomy file: otu_tax_Vsintax_unite.txt


In [69]:
# Number of OTUs
!grep -c "OTU_" $otutabout_file
!grep -c ">" $otu_file_noN
!grep -c "OTU_" $taxfile

3737
3737
3737


In [70]:
# total number of samples
sys.stdout.write('Total number of samples: ')
!head -n 1 $otutabout_file | perl -pe 's/[^\t]+\t//; s/\t/\n/g' | wc -l

Total number of samples: 182


# Playground
This is where code testing happens

In [154]:
!head -n 2 $qcFinalUniq

>2-3-L_4;size=4768318;
TTGTCGAAACCTGCAACAGCAGAACGACCCGTGAACACGTTTTAAACAGCTTGGGCGGGCGAGAGGAGCTTGCTCCTTGGACCCGCCCGCACCTGCTGGGAGAAATCTCGGCGGGCTAACGAACCCCGGCGCAATCTGCGCCAAGGAACAATAAAAGATTATCGCGTGGCTCGTGCGGTGGCCCGGAGACGGTGTCCGCCAATCGAGATGCGTGTTTATCGAAATGTCTAAACGACTCTCGGCAACGGATATCTCGGCTCTCGCNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
